In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

from matplotlib import rc
rc('font',family='AppleGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
from sklearn import linear_model
from scipy import stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import glob
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import ssl
import requests
from urllib.request import urlopen
from selenium import webdriver
import time
import re


In [122]:
### 각 연도별 데이터에서 필요 없는 행 열 제거 후 저장

def preprocessing(*yeardatas):
    for yeardata in yeardatas:
        data = pd.read_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Original_Movie_Data/Kofic_Original Data/KOBIS_연도별박스오피스_'+yeardata+'년도.csv')
        data = data.drop(index=[0, 1, 2], axis=0)
        data = data.rename(columns = data.iloc[0])
        data = data.drop(data.index[0])
        data = data.drop(columns = ['순위'], axis = 1)
        data = data.drop(columns = ['매출액 점유율'], axis = 1)
        data = data.drop(columns = ['국적'], axis = 1)
        data = data.reset_index(drop = True)
        data = data.drop(index = [len(data)-1], axis = 0)
        data = data.head(100)
        data.to_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Original_Movie_Data/Kofic_Merge Data/'+yeardata+'.csv',index=False)


In [123]:
preprocessing('2010', '2011', '2012', '2013','2014', '2015', '2016', '2017', '2018', '2019', '2020')

In [124]:
### 각 csv파일 합치기

input_data = '/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Kofic_movie_Data/Kofic_Merge Data'
output_data = '/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.csv'

data_list = glob.glob(os.path.join(input_data, '20*'))
all_data=[]
for file in data_list:
    df = pd.read_csv(file) 
    all_data.append(df)

combinedata = pd.concat(all_data, axis=0, ignore_index=True)
combinedata = combinedata.drop_duplicates(['영화명'])
combinedata.reset_index(drop = True,inplace = True)

In [132]:
### 저장

combinedata.to_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.csv',index=False)
combinedata.to_excel('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.xlsx',index=False)


In [134]:
### 불러올 때 매출액, 관객수, 스크린수, 상영횟수 int형 변환 
combinedata = pd.read_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.csv',thousands=',')
combinedata

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사
0,남산의 부장들,2020.1.22,41225216650,4750345,1659,140051,한국,(주)쇼박스
1,다만 악에서 구하소서,2020.8.5,38602260990,4357803,1998,193842,한국,(주)씨제이이엔엠
2,반도,2020.7.15,33072531280,3812250,2575,199048,한국,(주)넥스트엔터테인먼트월드(NEW)
3,히트맨,2020.1.22,20614278000,2406232,1122,87782,한국,롯데컬처웍스(주)롯데엔터테인먼트
4,테넷,2020.8.26,18396929850,1992214,2228,164758,미국,워너브러더스 코리아(주)
...,...,...,...,...,...,...,...,...
1042,파이널 데스티네이션5,2011.9.8,2370348500,257193,168,9325,미국,워너브러더스 코리아(주)
1043,카우보이 & 에이리언,2011.8.11,1930249000,256346,269,8244,미국,(주)씨제이이엔엠
1044,마마,2011.6.1,1642407500,243209,269,12667,한국,롯데쇼핑㈜롯데엔터테인먼트
1045,적과의 동침,2011.4.27,1740340000,243094,391,14069,한국,(주)쇼박스


In [135]:
combinedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047 entries, 0 to 1046
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영화명     1047 non-null   object
 1   개봉일     1047 non-null   object
 2   매출액     1047 non-null   int64 
 3   관객수     1047 non-null   int64 
 4   스크린수    1047 non-null   int64 
 5   상영횟수    1047 non-null   int64 
 6   대표국적    1047 non-null   object
 7   배급사     1047 non-null   object
dtypes: int64(4), object(4)
memory usage: 65.6+ KB


In [69]:
combinedata['국적'].value_counts()

한국             455
미국             418
일본              34
미국,영국           19
프랑스             10
              ... 
미국,뉴질랜드,벨기에      1
한국,체코            1
미국,영국,스페인        1
미국,영국,호주         1
한국,미국,홍콩         1
Name: 국적, Length: 65, dtype: int64

In [8]:
movie_list=combinedata['영화명']
movie_list

0                   극한직업
1                     명량
2             어벤져스: 엔드게임
3                    베테랑
4              신과함께-인과 연
              ...       
1042    더 프린세스 : 도둑맞은 공주
1043            프리키 데스데이
1044              날씨의 아이
1045              눈의 여왕4
1046              다크 나이트
Name: 영화명, Length: 1047, dtype: object

In [21]:
### 영화 이름, 장르 , 감독, 주연배우, 영화등급 crawling 

name_list = []
genre_list = []
director_list = []
actor_list = []
film_rank_list = []


In [22]:
## 특정 url로 request하거나 urlopen시 아래와 같은 에러가 발생하는 경우해결
ssl._create_default_https_context = ssl._create_unverified_context

driver = webdriver.Chrome('/Users/hanjaemin/PycharmProjects/acorn/chromedriver')
driver.get("https://movie.naver.com/")

In [23]:
for movie in movie_list:
    elem = driver.find_element_by_id("ipt_tx_srch")
    elem.send_keys(movie)
    time.sleep(1)
    try:
        path = driver.find_element_by_xpath('//*[@id="jAutoMV"]/ul/li[1]/a')
        path.click()
        
    except:
        elem.send_keys(Keys.RETURN)
        driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a').click()

    name = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')

    genre = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(1) > a:nth-child(1)')

    actor = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p')

    director = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4) > p > a')

    film_rank = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(8) > p')


    if len(name) == 0 :
        name_list.append('')
    else:
        for i in range(len(name)):
            name_list.append(name[i].text)


    if len(genre) == 0 :
        genre_list.append('')
    else:
        for i in range(len(genre)):
            genre_list.append(genre[i].text)


    if len(actor) == 0 :
        actor_list.append('')
    else:
        for i in range(len(actor)):
            actor_list.append(actor[i].text)


    if len(director) == 0 :
        director_list.append('')
    else:
        for i in range(len(director)):
            if i == 1:
                break
            director_list.append(director[i].text)


    if len(film_rank) == 0 :
        film_rank_list.append('')
    else:
        for i in range(len(film_rank)):
            film_rank_list.append(film_rank[i].text)




In [24]:
len(film_rank_list)

1047

In [41]:
len(name_list)

1047

In [26]:
len(genre_list)

1047

In [27]:
len(director_list)

1047

In [28]:
len(genre_list)

1047

In [29]:
len(actor_list)

1047

In [49]:
data={'director':director_list,
      'genre':genre_list,
      'actor':actor_list,
      'filmrank':film_rank_list}
crawling_data=pd.DataFrame(data)

In [50]:
crawling_data

,director,genre,actor,filmrank
0,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가
...,...,...,...,...
1042,올레 말라므,애니메이션,"사문영(밀라 목소리), 남도형(루슬란 목소리), 서반석(레스터 목소리)",[국내] 전체 관람가
1043,크리스토퍼 랜던,코미디,"빈스 본, 캐서린 뉴튼",[국내] 15세 관람가
1044,신카이 마코토,애니메이션,"심규혁(호다카), 김유림(히나), 최한(스가)",[국내] 15세 관람가
1045,로버트 렌스,애니메이션,"양정화(눈의 여왕 한국어 목소리), 박지윤(겔다 한국어 목소리), 임채헌(헤럴드 한...",[국내] 전체 관람가


In [51]:
crawling_data.to_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Naver_Crawling_Data/crawling_data.csv',index=False)
crawling_data.to_excel('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Naver_Crawling_Data/crawling_data.xlsx',index=False)

In [138]:
crawling_data = pd.read_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Naver_Crawling_Data/crawling_data.csv')
crawling_data

,director,genre,actor,filmrank
0,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가
...,...,...,...,...
1042,올레 말라므,애니메이션,"사문영(밀라 목소리), 남도형(루슬란 목소리), 서반석(레스터 목소리)",[국내] 전체 관람가
1043,크리스토퍼 랜던,코미디,"빈스 본, 캐서린 뉴튼",[국내] 15세 관람가
1044,신카이 마코토,애니메이션,"심규혁(호다카), 김유림(히나), 최한(스가)",[국내] 15세 관람가
1045,로버트 렌스,애니메이션,"양정화(눈의 여왕 한국어 목소리), 박지윤(겔다 한국어 목소리), 임채헌(헤럴드 한...",[국내] 전체 관람가


In [139]:
final_data = pd.concat([combinedata, crawling_data], axis=1)

In [142]:
### 열이름 변경
final_data.rename(columns={'영화명': 'moviename',
                          '개봉일' : 'openingdate',
                          '매출액' : 'sales',
                          '관객수' : 'attendance',
                          '스크린수' : 'screen',
                          '상영횟수' : 'playcount',
                          '대표국적' : 'country',
                          '배급사' : 'distributor'},inplace = True)

In [149]:
### 누적매출액 순으로 정렬

final_data.sort_values(['sales'], ascending = False, inplace = True)
final_data.reset_index(drop = True, inplace=True)
final_data

,moviename,openingdate,sales,attendance,screen,playcount,country,distributor,director,genre,actor,filmrank
0,극한직업,2019.1.23,139651845516,16265618,2003,292584,한국,(주)씨제이이엔엠,데이빗 예이츠,모험,"다니엘 래드클리프(해리 포터), 엠마 왓슨(헤르미온느), 루퍼트 그린트(론 위즐리)",[국내] 전체 관람가 [해외] PG-13
1,명량,2014.7.30,135748398910,17613682,1587,188611,한국,(주)씨제이이엔엠,벤 스타센,애니메이션,"빌리 웅거(새미), 팻 캐롤, 카를로스 맥쿨러스 2세(레이)",[국내] 전체 관람가
2,어벤져스: 엔드게임,2019.4.24,122182694160,13934592,2835,242001,미국,월트디즈니컴퍼니코리아 유한책임회사,민규동,멜로/로맨스,"임수정(연정인), 이선균(이두현), 류승룡(장성기)",[국내] 15세 관람가
3,베테랑,2015.8.5,105168155250,13414009,1115,199231,한국,(주)씨제이이엔엠,김정훈,멜로/로맨스,"이선균(정배), 최강희(다림)",[국내] 청소년 관람불가
4,신과함께-인과 연,2018.8.1,102666146909,12274996,2235,179981,한국,롯데컬처웍스(주)롯데엔터테인먼트,정범식,공포,"위하준(하준), 박지현(지현), 오아연(아연)",[국내] 15세 관람가
...,...,...,...,...,...,...,...,...,...,...,...,...
1042,더 프린세스 : 도둑맞은 공주,2020.8.12,457449660,55828,423,4192,우크라이나,(주)팝엔터테인먼트,황동혁,드라마,"공유(강인호), 정유미(서유진)",[국내] 청소년 관람불가
1043,프리키 데스데이,2020.11.25,451225690,51812,631,12882,미국,유니버설픽쳐스인터내셔널 코리아(유),브라이언 싱어,SF,"휴 잭맨(로건 / 울버린), 제임스 맥어보이(찰스 자비에), 마이클 패스벤더(에릭 랜셔)",[국내] 12세 관람가
1044,날씨의 아이,2019.10.30,444956180,58968,194,5921,일본,"메가박스중앙(주)플러스엠,(주)미디어캐슬,워터홀컴퍼니(주)",바이론 하워드,애니메이션,"지니퍼 굿윈(주디 홉스 목소리), 제이슨 베이트먼(닉 와일드 목소리), 샤키라(가젤...",[국내] 전체 관람가 [해외] PG
1045,눈의 여왕4,2019.12.24,444649620,67159,271,2239,러시아,(주)넥스트엔터테인먼트월드(NEW),홍원찬,범죄,"황정민(인남), 이정재(레이), 박정민(유이)",[국내] 15세 관람가


In [45]:
final_data.to_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Original_Movie_Data/merge_data.csv',index=False)
final_data.to_excel('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Original_Movie_Data/merge_data.xlsx',index=False)